If you want to be in the `PedestrianAvoidancePOMDP` environment, run the following cell. If the package has been added to the main julia environment, it is not necessary.

In [1]:
] activate ../ 

In [14]:
using Revise
using AutomotiveDrivingModels
using AutoViz
using AutomotiveSensors
using AutomotivePOMDPs
using GridInterpolations

using PedestrianAvoidancePOMDP
using POMDPPolicies

using JLD2
using FileIO
using Reel
using Random
using POMDPs
using POMDPModelTools
using LinearAlgebra

In [15]:
policy = load("../policy/policy.jld2")["policy"]; # need to be updated

In [22]:
pomdp = SingleOCFPOMDP()
updater = SingleOCFUpdater(pomdp)

parameters = CrosswalkParams()
parameters.roadway_length = 400.0
parameters.obstacles_visible = false

obstacle_offset = -2
obstacle_1 = ConvexPolygon([VecE2(34, obstacle_offset), VecE2(34, obstacle_offset-3), VecE2(46.5, obstacle_offset-3), VecE2(46.5, obstacle_offset)],4)
obstacle_2 = ConvexPolygon([VecE2(34, +4.5), VecE2(34, +7.5), VecE2(46.5, +7.5), VecE2(46.5, +4.5)],4)
parameters.obstacles = [obstacle_1, obstacle_2]
#parameters.obstacles = [obstacle_2]

env = CrosswalkEnv(parameters)
pomdp.env = env


ego_id = 1
ped_id = 2
ped2_id = 3


ego_v = 6.11
ped_v = 5.4/3.6
hitpoint = 50             # 50: middle, 100: left corner, 0: right corner
ped_theta = π/2


# fix values
ped_x = 50.0
ped_y_start = -5.0
ego_y = 0.0


ped_y_offset_hitpoint = VehicleDef().width * hitpoint / 100 - VehicleDef().width/2
ped_y_end = ped_y_offset_hitpoint
ped_t_collision = (ped_y_end - ped_y_start) / ped_v; 
ego_x = ped_x - ego_v * ped_t_collision - VehicleDef().length/2;

if ( false )
    ego_v = 0.0
    ego_x = 20
    ped_x = 30
    ped_y_start = -2.0
    ped_v = 0.0
end

ped_y_start = 0.0
ped_v = 0.0

# Car definition
ego_initial_state = VehicleState(VecSE2(ego_x, ego_y, 0.), env.roadway.segments[1].lanes[1], env.roadway, ego_v)
ego = Vehicle(ego_initial_state, VehicleDef(), 1)

# Pedestrian definition 
ped_initial_state = VehicleState(VecSE2(ped_x,ped_y_start,ped_theta), env.crosswalk, env.roadway, ped_v)
ped = Vehicle(ped_initial_state, AutomotivePOMDPs.PEDESTRIAN_DEF, 2)

ped2_v = 0.0
ped2 = Vehicle(VehicleState(VecSE2(54., 5., 0.78), env.crosswalk, env.roadway, ped2_v), AutomotivePOMDPs.PEDESTRIAN_DEF, ped2_id)


scene = Scene()
push!(scene, ego)
push!(scene, ped)
push!(scene, ped2)



cam = FitToContentCamera(0.);
timestep = 0.2


pos_noise = 0.001
vel_noise = 0.000
false_positive_rate = 0.00
false_negative_rate = 0.00
rng = MersenneTwister(1);
sensor = AutomotiveSensors.GaussianSensor(AutomotiveSensors.LinearNoise(10, pos_noise, 0.00), 
                 AutomotiveSensors.LinearNoise(10, vel_noise, 0.00), false_positive_rate, false_negative_rate, rng) 


# define a model for each entities present in the scene
models = Dict{Int, DriverModel}()
models[ego_id] = FrenetPedestrianPOMDP(a=LatLonAccel(0.0, 0.0),
    env=env,
    sensor=sensor,
    obstacles=env.obstacles,
    timestep=timestep,
    pomdp=pomdp,
    policy=policy,
    updater=updater,
    ego_vehicle=ego,
    desired_velocity=ego_v,
    b=PedestrianAvoidancePOMDP.initBeliefAbsentPedestrian(pomdp, ego_y, ego_v)
)
models[ego_id].pomdp.desired_velocity = ego_v
models[ego_id].pomdp.ΔT = timestep

models[ped_id] = ConstantPedestrian(v_desired=ped.state.v, dawdling_amp=0.001) # dumb model
models[ped2_id] = ConstantPedestrian(v_desired=ped2.state.v, dawdling_amp=0.001) # dumb model



nticks = 50
rec = SceneRecord(nticks+1, timestep)

risk = Float64[]
sensor_observations = [Vehicle[]]
belief = Dict{Int64, SingleOCFBelief}[]
ego_vehicle = Vehicle[]
action_pomdp = SingleOCFAction[]

obs_callback = (ObservationCallback(risk,sensor_observations, belief, ego_vehicle, action_pomdp),)

@elapsed begin
simulate!(rec, scene, env.roadway, models, nticks, obs_callback)
end




--------------------------POMDP high level planner----------------------- t: 0.0
EGO: x/y:27.633333333333333 / 0.0 v: 6.11
Vehicle(2, VehicleState(VecSE2({50.000, -0.001}, 1.571), Frenet(RoadIndex({1, 0.125001}, {1, 1}), 50.000, -0.001, 1.571), 0.000), VehicleDef(PEDESTRIAN, 1.000, 1.000))
Vehicle(3, VehicleState(VecSE2({54.000, 5.002}, 0.780), Frenet(RoadIndex({1, 0.135001}, {1, 2}), 54.000, 2.002, 0.780), 0.000), VehicleDef(PEDESTRIAN, 1.000, 1.000))
action_values: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
action_index_max: (0.0, 1)
p.action_map[ai[2]]: [1.0, 1.0]
action combined: [1.0, 1.0]
--------------------------POMDP high level planner----------------------- t: 0.2
EGO: x/y:28.875333333333334 / 0.2 v: 6.3100000000000005
Vehicle(2, VehicleState(VecSE2({50.000, 0.001}, 1.571), Frenet(RoadIndex({1, 0.125001}, {1, 1}), 50.000, 0.001, 1.571), -0.500), VehicleDef(PEDESTRIAN, 1.000, 1.000))
Vehicle(3, VehicleState(VecSE2({53.999, 4.999}, 0.780), Frene

b-length: 96
b-length: 79
action_values: [-3.02326, -0.629804, -3.41576, -5.12121, -8.16363, -1.96681, 0.0, -2.53018, -4.11991, -5.44781, -3.76077, -1.49698, -3.76077, -5.5901, -9.05135]
action_index_max: (0.0, 7)
p.action_map[ai[2]]: [0.0, 0.0]
action combined: [0.0, 0.0]
--------------------------POMDP high level planner----------------------- t: 2.6
EGO: x/y:46.81933333333336 / 1.0 v: 8.31
Vehicle(2, VehicleState(VecSE2({50.000, -0.499}, 1.571), Frenet(RoadIndex({1, 0.124999}, {1, 1}), 50.000, -0.499, 1.571), 0.000), VehicleDef(PEDESTRIAN, 1.000, 1.000))
Vehicle(3, VehicleState(VecSE2({53.999, 4.788}, 0.780), Frenet(RoadIndex({1, 0.134998}, {1, 2}), 53.999, 1.788, 0.780), 0.500), VehicleDef(PEDESTRIAN, 1.000, 1.000))
b-length: 77
b-length: 87
action_values: [-4.4011e-6, 0.0, -1.15966e-5, -4.23155e-5, -9.91467e-5, 0.0, 0.0, 0.0, -1.73918e-5, -4.66908e-5, -1.34221e-5, 0.0, -1.34221e-5, -4.41558e-5, -0.00011058]
action_index_max: (0.0, 2)
p.action_map[ai[2]]: [0.0, 1.0]
action combined

EGO: x/y:82.51533333333333 / 1.0 v: 11.709999999999988
Vehicle(2, VehicleState(VecSE2({50.000, -0.600}, 1.571), Frenet(RoadIndex({1, 0.125000}, {1, 1}), 50.000, -0.600, 1.571), 0.000), VehicleDef(PEDESTRIAN, 1.000, 1.000))
Vehicle(3, VehicleState(VecSE2({54.000, 4.929}, 0.780), Frenet(RoadIndex({1, 0.135000}, {1, 2}), 54.000, 1.929, 0.780), 0.500), VehicleDef(PEDESTRIAN, 1.000, 1.000))
action_values: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
action_index_max: (0.0, 1)
p.action_map[ai[2]]: [1.0, 1.0]
action combined: [1.0, 1.0]
--------------------------POMDP high level planner----------------------- t: 6.400000000000003
EGO: x/y:84.87733333333333 / 1.0 v: 11.909999999999988
Vehicle(2, VehicleState(VecSE2({49.998, -0.601}, 1.571), Frenet(RoadIndex({1, 0.124995}, {1, 1}), 49.998, -0.601, 1.571), 0.000), VehicleDef(PEDESTRIAN, 1.000, 1.000))
Vehicle(3, VehicleState(VecSE2({53.999, 4.999}, 0.780), Frenet(RoadIndex({1, 0.134998}, {1, 2}), 53.999, 1.999, 0.7

4.316584801

In [23]:
# Visualize scenario
duration, fps, render_hist = animate_record(rec, timestep, env, sensor, sensor_observations, risk, belief, ego_vehicle, action_pomdp, CarFollowCamera(2,10.0))
film = roll(render_hist, fps = fps, duration = duration)
#write("file.mp4", film) # Write to a webm video

Frames{MIME{Symbol("image/png")}}("/tmp/tmpcJN3N6", 0x0000000000000033, 5.0, nothing)